# Final Version of Phydra GEKKO Core v1
this is the very basis of core processes that was used for the first published version

can be used to develop the framework further in a smaller test environment, in these early versions

# TODO:
# 1. add value storage, both in Env and Comp
# 2. add a simple flux
# 3. test dim init: within Comp 1D (, within Env, flexible)

#IDEA:

remove the ENVS for noW! none of my use cases actually employ it!
so it would be awkward to present it there like that!!!

instead, just basic 0D model, but clean interface should be the goal!

in effect, as long as I use m.Array instead of m.SV, I should be able to add any kind of dims later on!

In [1]:
from gekko import GEKKO
import xsimlab as xs
import numpy as np

# to create dynamic storage of fluxes
from collections import defaultdict

# to measure process time
from time import process_time

In [255]:
# a function to randomly generate simple string as label for 
    # xsimlab dimension that should be flexible (scalar or list)
    
import string
import random

def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

In [340]:
@xs.process
class GekkoCore:
    m = xs.any_object()
    
    def initialize(self):
        print('initializing model core')
        self.m = GEKKO(remote=False, name='phydra')
        
        # add defaultdict of list that dynamically stores fluxes by component label
        self.m.fluxes = defaultdict(list)
        
    def finalize(self):
        print('finalizing gekko core: cleanup')
        #self.m.open_folder()
        self.m.cleanup()

        
@xs.process
class GekkoContext:
    """ Inherited by all other model processes to access GekkoCore"""
    m = xs.foreign(GekkoCore, 'm')

@xs.process
class GekkoSolve(GekkoContext):
    
    def finalize_step(self):
        print(self.m.__dict__)
        print('Running solver now')
        self.m.options.REDUCE = 3
        self.m.options.NODES = 3
        self.m.options.IMODE = 7
        
        solve_start = process_time()
        self.m.solve(disp=False)  # disp=True) # to print gekko output
        solve_end = process_time()

        print(f"ModelSolve done in {round(solve_end-solve_start,2)} seconds")
        
        print(self.m.__dict__)
        
@xs.process
class Time(GekkoContext):
    
    days = xs.variable(dims='time', description='time in days')
    # for indexing xarray IO objects
    time = xs.index(dims='time', description='time in days')

    def initialize(self):
        print('Initializing Model Time')
        self.time = self.days

        # ASSIGN MODEL SOLVING TIME HERE:
        self.m.time = self.time
        
        # add variable keeping track of time within model:
        self.m.timevar = self.m.Var(0, lb=0)
        self.m.Equation(self.m.timevar.dt() == 1)

        
@xs.process
class StateVariable(GekkoContext):
    label = xs.variable(intent='out')
    
    value = xs.variable(intent='out', dims='time', description='stores the value of component state variable')
    SV = xs.any_object(description='stores the gekko variable')
    
    initVal = xs.variable(intent='in', description='initial value of component')
    
    def initialize(self):
        self.label = self.__xsimlab_name__
        print(f"component {self.label} is initialized")
        
        self.SV = self.m.SV(self.initVal, name=self.label)
        self.value = self.SV.value
    
    def run_step(self):
        print('assembling Equations')
        #print(self.label, self.value)
        print(self.m.fluxes)
        self.m.Equation(self.SV.dt() == sum([flux for flux in self.m.fluxes[self.label]]))
        
    def finalize_step(self):
        pass

def monod(Input, Output, params={'halfsat':0.5}):
    return Input / (params['halfsat'] + Input) * Output
# now try to use this above in the flux below, and add func group dim parameter handling, that's intuitive!

@xs.process
class Fluxes(GekkoContext):
    label = xs.variable(intent='out')
    
    components = xs.variable(intent='in', dims=id_generator())
    
    def initialize(self):
        self.label = self.__xsimlab_name__
        print(f"flux {self.label} for {self.components} is initialized")
        
        flux = self.m.Param(0.1)
        for component in self.components:
            self.m.fluxes[component].append(flux)

In [341]:
a = xs.Model({'core':GekkoCore, 'solver':GekkoSolve, 'time':Time, 'N':StateVariable, 'P':StateVariable, 'growth':Fluxes})

In [342]:
in_ds = xs.create_setup(model=a,
                        clocks={'clock': [0,1]},
                        input_vars={
                            'time__days': ('time', np.arange(0, 1, 0.1)),
                            
                            'N':{'initVal':1},
                            'P':{'initVal':1},
                            'growth':{'components':['N','P']}
                        
                        },
                        output_vars={
                            'N__value':None,
                            'P__value':None
                        
                        })

In [343]:
with a:
    out_ds = in_ds.xsimlab.run()

initializing model core
Initializing Model Time
component N is initialized
component P is initialized
flux growth for ['N' 'P'] is initialized
assembling Equations
defaultdict(<class 'list'>, {'N': [0.1], 'P': [0.1]})
assembling Equations
defaultdict(<class 'list'>, {'N': [0.1], 'P': [0.1]})
{'_remote': False, '_server': 'http://byu.apmonitor.com', 'options': <gekko.gk_global_options.GKGlobalOptions object at 0x11673cdf0>, '_id': 39, '_gui_open': False, '_constants': [], '_parameters': [0.1], '_variables': [0, 1, 1], '_intermediates': [], '_inter_equations': [], '_equations': [<gekko.gekko.EquationObj object at 0x1156d3fd0>, <gekko.gekko.EquationObj object at 0x115b0fee0>, <gekko.gekko.EquationObj object at 0x115f0e070>], '_objectives': [], '_connections': [], '_objects': [], '_compounds': [], '_raw': [], 'time': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]), '_model_initialized': False, '_csv_status': None, '_model': '', '_model_name': 'phydra', '_path': '/var/folders/s8/8

In [344]:
out_ds

<xarray.Dataset>
Dimensions:             (BHWE2T: 2, clock: 2, time: 10)
Coordinates:
  * clock               (clock) int64 0 1
  * time                (time) float64 0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9
Dimensions without coordinates: BHWE2T
Data variables:
    N__initVal          int64 1
    N__value            (time) float64 1.0 1.01 1.02 1.03 ... 1.07 1.08 1.09
    P__initVal          int64 1
    P__value            (time) float64 1.0 1.01 1.02 1.03 ... 1.07 1.08 1.09
    growth__components  (BHWE2T) <U1 'N' 'P'
    time__days          (time) float64 0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9

# for later reference, for higher dimensionality:

In [163]:
def initialize_array(gk_class, label, shape, initval = None):
    """This function returns a labeled multi-dimensional m.array of supplied base gk_class"""
    gk_array = m.Array(gk_class, shape)
    
    it = np.nditer(gk_array, flags=['refs_ok','multi_index'], op_flags=['readwrite'])

    i=0
    while not it.finished:
        gk_array[it.multi_index].__dict__['NAME'] = f"{label}_{str(i)}"
        
        if initval != None:
            gk_array[it.multi_index].VALUE = initval
            
        i += 1
        #print(it.value)
        it.iternext()
        
    return gk_array

In [90]:
x_array = initialize_array(m.SV, 'Nutrient', (3,3,3), initval=0)